# Crystalization Notes

## 2017-10-09
Should have started this weeks ago.  Anyway, the idea is that we can use a variant of [this PPT by Cho](http://web.ecs.baylor.edu/faculty/cho/Cho-Entropy.pdf) to subgraphs of minimal energy.  

The ultimate goal is to take a patients CUI set and find the maximally fitting set of conditions for them.  The conditions should initially be informed by the "Conditions" listed with the CT.gov dataset, but at some point we'll let users construct their own.


### Recipe Reconstruction
The [Yummly Data](https://www.kaggle.com/c/whats-cooking) seems a nice analogy.  They provide a three-tiered data set that includes "Cuisine > Recipe > Ingredients".  [This guy](https://github.com/Btibert3/kaggle-yummly) tried it in Neo4j. The Recipe is just an idea. The intent of the competition was to be able to predict the cuisine given just the ingredients.  However, in our analogy I want to find the "maximally fitting" recipe for a given set of ingredients.  We don't have labels for this, so I don't have the success metric yet.

I took a set of about 124 recipes to run the minimal energy routine on.  Unfortunately, I messed up the code and I mistook it for not doing the minimization.  Specifically, I we returning the size of `cupboard` which is just the set of ingredient ids instead of returning the output of `minEnergy`.  To solve the problem I didn't have, I created the `knockout` routine, which removes a few ingredients from each recipe before it gets sent in for minimization.

The `knockout` routine may or may not be causing very small crystals.  I'm running some metrics on that now.  However, `knockout` isn't necessary right now since the error never really existed.

I think one of the issues it that the crystals are small because the data set is not very connected. In the 124 set, we don't get much coverage of any one cuisine, and though I'm not sure how to test it, I'm thinking highly specific ingredients are experiencing very low degrees.

I still feel confident the method will be informative, but it's not being as cooperative as I'd like.

Today, I think I'm going to try to extract all "Italian" recipes or something like that.  Actually, I should probably pull the whole thing into Postgres so I can sub-set as needed.

```
DROP TABLE IF EXISTS r.yummly;
CREATE TABLE r.yummly
(cuisine text
, recipe int
, ingredient text
);
\copy r.yummly from yummly.txt 
```

### SQL EDA

```
select cuisine, count(distinct(recipe)), count(distinct(ingredient)) 
from r.yummly 
group by 1 
order by 1;

   cuisine    | count | count
--------------+-------+-------
 brazilian    |   467 |   853
 british      |   804 |  1166
 cajun_creole |  1546 |  1576
 chinese      |  2673 |  1792
 filipino     |   755 |   947
 french       |  2646 |  2102
 greek        |  1175 |  1198
 indian       |  3003 |  1664
 irish        |   667 |   999
 italian      |  7838 |  2929
 jamaican     |   526 |   877
 japanese     |  1423 |  1439
 korean       |   830 |   898
 mexican      |  6438 |  2684
 moroccan     |   821 |   974
 russian      |   489 |   872
 southern_us  |  4320 |  2462
 spanish      |   989 |  1263
 thai         |  1539 |  1376
 vietnamese   |   825 |  1108
```

```
SELECT ingredient
, count(distinct(cuisine)) AS n_cuisines
, count(*) AS n_records
FROM r.yummly
GROUP BY 1
ORDER BY 2 DESC, 3 DESC, 1
LIMIT 35;

       ingredient       | n_cuisines | n_records
------------------------+------------+-----------
 salt                   |         20 |     18049
 olive oil              |         20 |      7972
 onions                 |         20 |      7972
 water                  |         20 |      7457
 garlic                 |         20 |      7380
 sugar                  |         20 |      6434
 garlic cloves          |         20 |      6237
 butter                 |         20 |      4848
 ground black pepper    |         20 |      4785
 all-purpose flour      |         20 |      4632
 pepper                 |         20 |      4438
 vegetable oil          |         20 |      4385
 eggs                   |         20 |      3388
 soy sauce              |         20 |      3296
 kosher salt            |         20 |      3113
 green onions           |         20 |      3078
 tomatoes               |         20 |      3058
 large eggs             |         20 |      2948
 carrots                |         20 |      2814
 unsalted butter        |         20 |      2782
 extra-virgin olive oil |         20 |      2747
 black pepper           |         20 |      2627
 milk                   |         20 |      2263
 oil                    |         20 |      1970
 red bell pepper        |         20 |      1939
 purple onion           |         20 |      1896
 scallions              |         20 |      1891
 ginger                 |         20 |      1755
 baking powder          |         20 |      1738
 jalapeno chilies       |         20 |      1730
 fresh lemon juice      |         20 |      1679
 minced garlic          |         20 |      1583
 chicken broth          |         20 |      1554
 cayenne pepper         |         20 |      1523
 brown sugar            |         20 |      1503
```


```
SELECT ingredient
, count(distinct(cuisine)) AS n_cuisines
, count(*) AS n_records
FROM r.yummly
GROUP BY 1
ORDER BY 2, 3, 1
LIMIT 35;

               ingredient                | n_cuisines | n_records
-----------------------------------------+------------+-----------
 2% low fat cheddar chees                |          1 |         1
 2% milk shredded mozzarella cheese      |          1 |         1
 33% less sodium cooked deli ham         |          1 |         1
 33% less sodium cooked ham              |          1 |         1
 33% less sodium smoked fully cooked ham |          1 |         1
 8 ounc ziti pasta, cook and drain       |          1 |         1
 abbamele                                |          1 |         1
 absinthe                                |          1 |         1
 acai juice                              |          1 |         1
 accompaniment                           |          1 |         1
 Adobo All Purpose Seasoning             |          1 |         1
 adobo style seasoning                   |          1 |         1
 agar agar flakes                        |          1 |         1
 agave tequila                           |          1 |         1
 aged gouda                              |          1 |         1
 alaskan king salmon                     |          1 |         1
 aleppo                                  |          1 |         1
 Alexia Waffle Fries                     |          1 |         1
 almond oil                              |          1 |         1
 almond syrup                            |          1 |         1
 aloe juice                              |          1 |         1
 Amarena cherries                        |          1 |         1
 amba                                    |          1 |         1
 amberjack fillet                        |          1 |         1
 amber rum                               |          1 |         1
 america                                 |          1 |         1
 american cheese food                    |          1 |         1
 ammonium bicarbonate                    |          1 |         1
 amontillado sherry                      |          1 |         1
 and carrot green pea                    |          1 |         1
 and cook drain pasta ziti               |          1 |         1
 angled loofah                           |          1 |         1
 angus                                   |          1 |         1
 anise basil                             |          1 |         1
 anise liqueur                           |          1 |         1
(35 rows)

```


```
SELECT n_cuisines
, count(*)
FROM (
    SELECT ingredient
    , count(distinct(cuisine)) AS n_cuisines
    FROM r.yummly
    GROUP BY 1
) AS a
GROUP BY n_cuisines
ORDER BY 1,2;


 n_cuisines | count
------------+-------
          1 |  2597
          2 |  1009
          3 |   588
          4 |   460
          5 |   323
          6 |   257
          7 |   220
          8 |   177
          9 |   159
         10 |   127
         11 |   110
         12 |   100
         13 |    81
         14 |    76
         15 |    72
         16 |    61
         17 |    65
         18 |    57
         19 |    68
         20 |   107
(20 rows)
```